<a href="https://colab.research.google.com/github/hernansalinas/autogrades/blob/main/Laboratorios_Taller/Lab08_Derivadas_e_integrales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import sympy as sy

## Parte 1

En un cirtcuito con un voltaje de  $\varepsilon(t)$ y una inductancia de L, la primera ley de kirshhoff nos da la siguiente relación

\begin{equation}
\varepsilon(t) = L\frac{\mathrm{d}i}{\mathrm{d}t} + Ri
\end{equation}

donde $R$, es la resistencia del ciruito, $I$ es la corriente. Suponga que medimos la corriente con varios valores de $t$ y obtenemos:

``t = 1.00, 1.01, 1.02, 1.03, 1.04``

``i = 3.10, 3.12, 3.14, 3.18, 3.24``


donde $t$ se mide en segundos, $I$ se da en Amperios, la inductancia $L$ es una constante de 0.98 H, la resistencia es de 0.142 $\Omega$. Aproxime el votaje  $\varepsilon(t)$ en los valores t=1.00, 1.01, 1.02, 1.03, 1.04 ,  para ello realice lo siguiente:


- Construya un data frame de pandas con los valores de tiempo y corriente
- Emplee un metodo calcular la derivada numérica.

In [2]:
data={"t":[1.00, 1.01, 1.02, 1.03, 1.04], "i": [3.1, 3.12, 3.14, 3.18, 3.24]}

df=pd.DataFrame(data=data)
L=0.98
R=0.142

df["di"]=(df.i.shift(-1)-df.i.shift(1))/0.02
df.di[0]=(df.i[1]-df.i[0])/0.01
df.di[4]=(df.i[4]-df.i[3])/0.01
df["voltaje"]=L*df.di+R*df.i
df

,t,i,di,voltaje
0,1.00,3.10,2.0,2.40020
1,1.01,3.12,2.0,2.40304
2,1.02,3.14,3.0,3.38588
3,1.03,3.18,5.0,5.35156
4,1.04,3.24,6.0,6.34008


## Parte 2
1. Escribir en código Markdown la expresión para calcular una integral con la regla del trapecio.

2. Escribir en código Markdown la expresión para calcular una integral con la regla del simpson.

3. Escribir en código Markdown la expresión para calcular una integral con el metodo de cuadratura.

4. Calcular una aproximación con el metodo 1,2,3 de las siguientes integrales

### Punto 1
La regla del trapecio está dada por:
$$\int_a^b f(x)dx\approx(b-a)(\frac{f(a)+f(b)}2)$$
### Punto 2
La regla de simpson está dada por:
$$\int_a^bf(x)dx\approx\frac{b-a}6(f(b)+4f(\frac{b+a}2)+f(a))$$
### Punto 3
Si uno tiene un conjunto de nodos $x_0,x_1,...,x_n$ en el intervalo $[a, b]$ entonces
$$\int_a^bf(x)dx\approx\sum_{i=0}^nf(x_i)A_i$$
Con $A_i=\int_a^bl_i(x)dx$. Donde $l_i$ son los polinomios de interpolación de lagrange de los puntos $\{(x_i,f(x_i))\}_{i=0}^n$

In [4]:
# Programas aproximación de la integral

def Trapecio(X, Y):
    s=0
    for i in range(len(X)-1):
        s+=(X[i+1]-X[i])*(Y[i+1]+Y[i])/2
    return s

def Simpson(X, Y):
    if not(len(X)%2==1):
        return "Error, número de puntos par"
    s=0
    for i in range(int((len(X)-1)/2)):
        s+=((X[2*i+2]-X[2*i])/6)*(Y[2*i+2]+4*Y[2*i+1]+Y[2*i])
    return s

def Quadratura(X, Y):
    B=np.array([(1/(i+1))*(X[-1]**(i+1)-X[0]**(i+1)) for i in range(len(X))])
    print("B defined")
    A=np.array([[np.float128(y)**j for y in X] for j in range(len(X))])
    print(A)
    A=A.astype(np.float64)
    B=B.astype(np.float64)
    A=np.linalg.solve(A, B) #Obteniendo los A_i por el método de coeficientes indeterminados
    s=0
    for i in range(len(A)):
        s+=Y[i]*A[i]
    return(s)



---
$$
 \int_0^{\pi/2} \frac{1}{\sqrt{\tan x}}\mathrm{dx}
 $$

----
$$
 \int_{1}^{\infty} \frac{\cos x}{x^3}\mathrm{dx}
 $$


----

$$
 \int_{0}^{\pi/4} \int_{0}^{\sin x} \frac{1}{\sqrt{1-y^2}}\mathrm{dxdy}
 $$

---

Garantizando  que el error sea menor a $\varepsilon=1\times 10^{-5}$.

In [6]:
from scipy.integrate import trapezoid, simps, quad
from sympy import integrate

x=sy.Symbol("x")
y=sy.Symbol("y")
sf1=1/sy.sqrt(sy.tan(x))
sf2=sy.cos(x)/x**3
sf3=1/sy.sqrt(1-x**2)
sf4 = sy.integrate(sf3, (x, 0, sy.cos(y)))

sI1=integrate(sf1, (x, 0, sy.pi/2))

f1=sy.lambdify(x, sf1)
f2=sy.lambdify(x, sf2)
f3= sy.lambdify(x, sf3)

def f4(y):
    T=np.linspace(0, np.sin(y), 10001)
    return Simpson(T, f3(T))


X1=np.linspace(1e-6, np.pi/2, 10000001)
print(f"Aproximando f1 con el método del trapecio: {Trapecio(X1, f1(X1))}")
print(f"Aproximando f1 con el método de simpson: {Simpson(X1, f1(X1))}")

X2=np.linspace(1, 1000, 1000001)
print(f"Aproximando f2 con el método del trapecio: {Trapecio(X2, f2(X2))}")
print(f"Aproximando f2 con el método de simpson: {Simpson(X2, f2(X2))}")

X3=np.linspace(0, np.pi/2-1e-2, 10001)
print(f"Aproximando f3 con el método del trapecio: {Trapecio(X3, f4(X3))}")
print(f"Aproximando f3 con el método de simpson: {Simpson(X3, f4(X3))}")


Aproximando f1 con el método del trapecio: 2.2194424955777956
Aproximando f1 con el método de simpson: 2.219441475146155
Aproximando f2 con el método del trapecio: 0.018117827593677888
Aproximando f2 con el método de simpson: 0.01811762280610441
Aproximando f3 con el método del trapecio: 1.2180430207702744
Aproximando f3 con el método de simpson: 1.2180430206337935


## Parte 3
Un péndulo físico en el regimen de pequeñas oscilaciones tiene un periodo de:

\begin{equation}
P_0=2\pi \sqrt{\frac{I_0}{mgb}}
\end{equation}

Cuando las oscilaciones no son pequeñas, podemos encontrar una expresión para el periodo como sigue:


La energia de un péndulo físico viene dada por:

\begin{equation}
E=\frac{I_0\omega^2}{2}+mgb(1-\cos \theta)
\end{equation}

En el punto más alejado de su posición de equilibrio, su energía es solo potencial y viene dada por:

\begin{equation}
Ep = mgb(1-\cos \theta),
\end{equation}

Igualando ambas expresiones tenemos que:

\begin{equation}
mgb(1-\cos \theta)=\frac{I_0\omega^2}{2}+mgb(1-\cos \theta)
\end{equation}

Integrando la ecuación respecto a $\theta$ y realizando la sustitución:

$\sin \phi = \frac{\sin \frac{\theta}{2}}{\sin \frac{\theta_0}{2}}$ y  $k = \sin \frac{\theta_0}{2}$, se puede encontrar  que el periodo de oscilación, viene dado en terminos de  la integral elíptica completa de primera especie:


\begin{equation}
P = \frac{2P_0}{\pi} \int _0^{\pi/2} \frac{\mathrm{d\phi}}{\sqrt{1-k^2\sin^2\phi}}
\end{equation}

donde,  $P_0$ es el periodo de las oscilaciones pequeñas.

Determinar el periodo de oscilación de un péndulo. Para ello:

1. Suponga los párametros iniciales del pendulo.
2. Realice una buena planificación del código para calcular la integral.
3. Emplee un  método de integracion numérico que conozca.
4. Compruebe su solucion con el valor exacto de la integral mostrado en el punto

In [73]:
# Definiendo constantes
g=9.8
L=1
k=np.sin(np.pi/4)
P0=2*np.pi*np.sqrt(L/g)
o=sy.Symbol("0")

Ps=2*P0/np.pi*1/(sy.sqrt(1-k**2*sy.sin(o)**2))
Pn=sy.lambdify(o, Ps)
print(f"Valor exacto de la integral: {sy.integrate(Ps, (o, 0, sy.pi/2))}")
T=np.linspace(1e-3, np.pi/2, 1001)
print(f"Approximación numérica: {Simpson(T, Pn(T))}")


Valor exacto de la integral: 2.36904972217534
Approximación numérica: 2.3677719689388645


## Parte 4
The next  problems are examples of how elliptic integrals enter into realistic physics problems. It is straightforward to evaluate any integral numerically using the techniques of this chapter, but it may be difficult for you to  know if the answers you obtain are correct. One way to hone your integral evaluating skills is to compare your answers from quadrature to power series
expressions, or to a polynomial approximations of know precision. To help you in this regard, we present here a polynomial approximation for an elliptic integral (Abramowitz and Stegun, 1972):

\begin{equation}
K(m)=\int_{0}^{\pi/2} \frac{\mathrm{d\phi}}{\sqrt{1-m\sin^2\phi}}
\end{equation}

\begin{equation}
\approx a_0+a_1m_1+a_2m_2 - [ b_0+b_1m_1+b_2m_1^2]\ln m_1 + \epsilon(m)
\end{equation}


* $m_1 = 1-m$,

* $ 0 \le m \le1 $

* $\epsilon(m)\leq 3\times 10^{-5}$

* $a_0 = 1.3862944$

* $b_0 = 0.5$

* $a_1 = 0.1119723$

* $b_1 = 0.1213478$

* $a_2 = 0.00725296$

* $b_2 = 0.0288729$

In [78]:
m=1/2
m1=1-m
a0=1.3862944
a1=0.1119723
a2=0.00725296
b0=0.5
b1=0.1213478
b2=0.0288729

a0+a1*m1+a2*m1-(b0+b1*m1+b2*m1**2)*np.log(m1)

1.839539855305946

## Parte 5 (Opcional)

Figure 5.7 shows a current loop of radius a carrying a current I. The point P
is a distance r from the center of the loop with spherical coordinates (r, θ, φ).
Jackson (1988) solves for the φ component of the vector potential at point P
in terms of elliptic integrals:

\begin{equation}
A_{\phi}(r, \theta)=\frac{\mu_0}{4\pi}\frac{4Ia}{\sqrt{a^2+r^2+2ar\sin \theta}}\left(\frac{(2-k^2)K(k)-2E(k)}{k^2} \right)
\end{equation}


where

\begin{equation}
K(k)=\int_{0}^{\pi/2} \frac{\mathrm{d\phi}}{\sqrt{1-k^2\sin^2\phi}}
\end{equation}


\begin{equation}
E(k)=\int_{0}^{\pi/2} \mathrm{d\phi} \sqrt{1-k^2\sin^2\phi}
\end{equation}


\begin{equation}
k^2=\frac{4ar\sin\theta}{a^2+r^2+2ar\sin\theta}
\end{equation}



Here K(k) is a complete elliptic integral of the first kind and E(k) is a complete
elliptic integral of the second kind. For a = 1, I = 3, and μ0∕4π = 1, compute
and plot

a) Aφ(r = 1.1, θ) vs. θ

b) Aφ(r, θ = π∕3) vs. r
